In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = os.path.join(os.getcwd(), '.kaggle')
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_DISABLED"] = "true"

In [4]:
!kaggle datasets download -d xmaulana/psychikadataset-7b

Dataset URL: https://www.kaggle.com/datasets/xmaulana/psychikadataset-7b
License(s): unknown
  0% 0.00/2.18M [00:00<?, ?B/s]
100% 2.18M/2.18M [00:00<00:00, 587MB/s]


In [5]:
import zipfile
with zipfile.ZipFile('psychikadataset-7b.zip', 'r') as zip_ref:
    zip_ref.extractall('psychikadataset-7b')

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
import torch

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [7]:
# Cek device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Load model dan tokenizer
model_name = "Qwen/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Konfigurasi Scaled Dot-Product Attention jika diperlukan (optional)
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_math_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(False)

# Aktifkan gradient checkpointing
model.gradient_checkpointing_enable()
model.config.use_cache = False

# LoRA config
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["c_attn", "q_proj", "v_proj"],  # sesuaikan jika error
)

# Terapkan LoRA ke model
model = get_peft_model(model, peft_config)
model.to(device)

# Load dataset
df = pd.read_csv("psychikadataset-7b/data.csv")

# Format sesuai template chat Qwen
def format_chat_template(row):
    chat = [
        {"role": "user", "content": row["Context"]},
        {"role": "assistant", "content": row["Response"]}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

df["text"] = df.apply(format_chat_template, axis=1)

# Convert ke HuggingFace Dataset
dataset = Dataset.from_pandas(df[["text"]])

# Tokenisasi
def tokenize(sample):
    return tokenizer(sample["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./qwen-psychika-lora",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_steps=99999999,
    save_steps=100,
    save_total_limit=1,
    fp16=True,  # GUNAKAN fp16 untuk T4
    report_to=[],
    logging_strategy="no"
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator,
)

# Mulai training
trainer.train()

# Simpan model dan tokenizer
model.save_pretrained("qwen-psychika-lora")
tokenizer.save_pretrained("qwen-psychika-lora")

Device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

Map:   0%|          | 0/5667 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5667 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


('qwen-psychika-lora/tokenizer_config.json',
 'qwen-psychika-lora/special_tokens_map.json',
 'qwen-psychika-lora/chat_template.jinja',
 'qwen-psychika-lora/vocab.json',
 'qwen-psychika-lora/merges.txt',
 'qwen-psychika-lora/added_tokens.json',
 'qwen-psychika-lora/tokenizer.json')

In [8]:
!zip -r qwen-psychika-lora.zip qwen-psychika-lora

  adding: qwen-psychika-lora/ (stored 0%)
  adding: qwen-psychika-lora/special_tokens_map.json (deflated 61%)
  adding: qwen-psychika-lora/checkpoint-5667/ (stored 0%)
  adding: qwen-psychika-lora/checkpoint-5667/special_tokens_map.json (deflated 61%)
  adding: qwen-psychika-lora/checkpoint-5667/optimizer.pt (deflated 8%)
  adding: qwen-psychika-lora/checkpoint-5667/adapter_model.safetensors (deflated 8%)
  adding: qwen-psychika-lora/checkpoint-5667/rng_state.pth (deflated 25%)
  adding: qwen-psychika-lora/checkpoint-5667/chat_template.jinja (deflated 46%)
  adding: qwen-psychika-lora/checkpoint-5667/tokenizer_config.json (deflated 69%)
  adding: qwen-psychika-lora/checkpoint-5667/scaler.pt (deflated 60%)
  adding: qwen-psychika-lora/checkpoint-5667/merges.txt (deflated 57%)
  adding: qwen-psychika-lora/checkpoint-5667/scheduler.pt (deflated 55%)
  adding: qwen-psychika-lora/checkpoint-5667/training_args.bin (deflated 51%)
  adding: qwen-psychika-lora/checkpoint-5667/vocab.json (deflat

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer dan model yang sudah di-finetune
tokenizer = AutoTokenizer.from_pretrained("./qwen-psychika-lora", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./qwen-psychika-lora", trust_remote_code=True).to(device)
model.eval()

# Inisialisasi chat history
chat_history = [
    {"role": "system", "content": "Kamu adalah seorang profesional psikiater yang sedang memeriksa pasien yang sedang mengalami tekanan mental."}
]

print("Mulai percakapan dengan Qwen Psychika Chat (ketik 'exit' untuk keluar):")

# Loop percakapan
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    chat_history.append({"role": "user", "content": user_input})

    # Format prompt
    chat_prompt = tokenizer.apply_chat_template(chat_history, tokenize=False)

    # Tokenisasi dan pindahkan ke device
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Ambil token output baru saja (tanpa prompt)
    prompt_len = inputs["input_ids"].shape[1]
    generated_tokens = outputs[0][prompt_len:]
    reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    print("AI:", reply)

    chat_history.append({"role": "assistant", "content": reply})

Mulai percakapan dengan Qwen Psychika Chat (ketik 'exit' untuk keluar):
You: halo ai saya dengan ad  masalah
AI: assistant
Saya menemui beberapa orang yang mengerti bagaimana berbicara dengan mereka.   Saya perlu meminta bantuan untuk membantu Anda menangani masalah ini.   Ini adalah sesuatu yang sangat penting dan kami semua memiliki sumber daya yang dapat kita miliki untuk membantu kita melakukan hal itu.   Ada banyak cara,  tetapi ada yang paling baik bagi Anda adalah melihat apa yang Anda lakukan sekarang dan bagaimana membuatnya lebih sulit lagi.   Apakah Anda sedang merasa terlalu berdampak pada diri sendiri?   Apakah Anda mencoba memperbaikinya atau mengalami kesulitan dalam mengatasi masalah ini?   Sebagian besar orang yang terkait dengan masalah ini tidak akan pernah mendapatkan respon dari orang lain yang menulis pertanyaan.   Tidak ada hal yang lebih baik dibutuhkan dari Anda.   Karena Anda ingin memahami apa yang Anda lakukan secara konsisten,  maka Anda mungkin dapat mengg